# Example : Common Tensorflow workflow

## Summary
This example is common tensorflow training sequence.

On this example, we will train hand-writing number classification model with [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database).
- - -
### Import pacakges

In [ ]:
# This package will use for install uninstalled package
import subprocess

# Import TensorFlow and install if not installed
try:
    import tensorflow as tf
except(ModuleNotFoundError):
    subprocess.run(["pip3", "install", "tensorflow"])
    import tensorflow as tf

# Import NumPy and install if not installed
try:
    import numpy as np
except(ModuleNotFoundError):
    subprocess.run(["pip3", "install", "numpy"])
    import numpy as np

# Import OpenCV and install if not installed
try:
    import cv2
except(ModuleNotFoundError):
    subprocess.run(["pip3", "install", "opencv-python"])
    import cv2

# Import Python internal modules
import os
import glob
from pathlib import Path

#### What those packages do?
* [TensorFlow](https://www.tensorflow.org/) : Define and training model.
* [Numpy](https://numpy.org/) : Preprocess image array data for training data.
* [cv2 (OpenCV)](https://opencv.org/) : Read image file and return it to numpy array.
* [os](https://docs.python.org/3/library/os.html) : Get label and join splited path to one.
* [glob](https://docs.python.org/3/library/glob.html) : Get all image files absolute path.
* [pathlib](https://docs.python.org/3/library/pathlib.html) : type-hinting.
- - -
### Define dataset and dataloader
We will assume dataset is infinite or It can only stored partial dataset in [RAM](https://en.wikipedia.org/wiki/Random-access_memory). so we won't cache **All** dataset and preprocessed data in RAM.
[Every data will processed by inline](https://www.tensorflow.org/guide/data_performance).

dataflow will be like picture below: 

<img src="./imgs/without_gpudirect_storage.png" width='425px' height='450px'>

#### Preprocsesing step
1. Read image file from NVMe SSD
2. Decode image to NumPy arary
3. Normalize image by divide array to 255
4. Transform 'HW' array into 'CHW' array by Add Channel dimension to index 0 on array

In [2]:
def path_to_label(file_path: Path):
    """Return label from path

    Arg:
        file_path (Path): path of file

    Return:
        label (np.int32): label of file

    """
    
    _, label = os.path.split(os.path.dirname(file_path))
    return label

def path_to_data(file_path: Path):
    """Return Preprocessed data and from path

    Arg:
        file_path (Path): path of file

    Return:
        data (np.int32): preprocessed data of file

    """

    # Read image data and return Numpy Array
    data = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    # Normalize image
    data = data/255.
    # Expand dim 0 for batch training
    data = np.expand_dims(data, axis=0)

    return data

class MNISTDataset(tf.data.Dataset):
    def _generator(dir_path: Path):
        """Return data loading generator function

        Arg:
            dir_path (Path): Directory of dataset

        Yields:
            data (np.float32): preprocessed data
            label (np.int32): label of data
        """

        file_list = glob.glob(os.path.join(dir_path, '*/*.png'))
        for file_path in file_list:
            data = path_to_data(file_path)
            label = path_to_label(file_path)
            
            yield (data, label)
    
    def __new__(cls, image_dir:Path):
        """Return TensorFlow Dataset object from generator function

        Arg:
            cls (tf.data.Dataset): dataset classitselves
            image_dir (Path): Path to images

        Return:
            dataset (tf.data.Dataset) : dataset from generator
        """

        dataset = tf.data.Dataset.from_generator(
            lambda: cls._generator(image_dir),
            output_signature=(
            tf.TensorSpec(shape=(1, 28, 28), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32),
            ),
        )
        return dataset

### Define model, optimizer, loss function

This example Task is 'Multi labels classification'. so model would like below.

* Model is simple model Based on [Convolutional Layers](https://arxiv.org/abs/1511.08458).
* Loss function will be [sparse categorical crossentropy](https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy).
* Optimizer will be [AdamW](https://arxiv.org/abs/1711.05101).

For convenience, model's performance would be only measured by train set accuracy.

#### Model architecture
<img src="./imgs/model_architecture.png" width="300px" height="500px">

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=(1, 28, 28), activation='relu', data_format='channels_first'),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128)
])

optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

### Compile model and start training

Current Training Environment is like below:

|Precision|Batch preprocssing|Batch caching|GPU select|GPU memory strategy|
|---|---|---|---|---|
|[TF32 (TensorFloat-32)](https://blogs.nvidia.com/blog/2020/05/14/tensorfloat-32-precision-format/)|Inline<br>Compute by CPU|Cache only next batch<br>Stored in RAM|Automatically Selected by TensorFlow|As much as Possible<br>[(TensorFlow Default GPU memory strategy)](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)|


In [ ]:
epochs = 100
batch_size = 2560
images_path = './mnist_png/training'

# Define dataset for training
dataset = MNISTDataset(image_dir=images_path).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Compile model and sent to GPU memory
model.compile(optimizer=optimizer,
            loss=loss_fn,
            metrics=metrics)

# Training model
model.fit(dataset, epochs=epochs)